## **Argubot**: Un Cchatbot sobre la tesis doctoral de Tomás Manzur "CONSTRUCCIÓN DE ARGUMENTOS Y CONTROVERSIAS SOCIO-TÉCNICAS: Análisis de la conflictividad surgida en la discusión del Plan Provincial de Ordenamiento Territorial de la Provincia de Mendoza"

# Descripción del Chatbot "ArguBot": Herramienta Interactiva y Segura para Consultas sobre la Tesis Doctoral de Tomás Manzur

## Introducción
El **ArguBot** es un chatbot revolucionario, implementado en un **Jupyter Notebook**, diseñado para ofrecer asistencia especializada en consultas sobre la tesis doctoral de Tomás Manzur, "Construcción de Argumentos y Controversias Socio-Técnicas". Este bot se destaca por su análisis avanzado y su enfoque en la **privacidad del usuario** y el uso de **fuentes de código abierto**.

## Funcionalidades Clave

### Uso Exclusivo de Fuentes Open Source
ArguBot está construido con fuentes de código abierto, incluyendo:
- Modelo Gemini de Google
- 'llamaindex' para gestión de datos
- 'chromadb' para búsquedas eficientes
Esto asegura **transparencia** y **adaptabilidad**.

### Privacidad del Usuario
La interacción con el bot requiere una **API_KEY**, ingresada de manera segura mediante `getpass`. Esto garantiza la **confidencialidad** y la **protección de los datos del usuario**.

### Implementación en Jupyter Notebook
ArguBot aprovecha la flexibilidad del entorno Jupyter Notebook, permitiendo una implementación sencilla y personalizable, con altos estándares de seguridad.

### Funciones Integradas
- **Consulta de Contenidos Específicos:** Respuestas detalladas sobre la tesis.
- **Análisis de Argumentos:** Explica argumentos manteniendo la confidencialidad.
- **Contextualización del Conflicto:** Información segura sobre el Plan de Ordenamiento Territorial.
- **Recursos Adicionales:** Enlaces a publicaciones y materiales relacionados que respetan la privacidad.

## Aplicación Práctica
ArguBot es una herramienta interactiva y educativa avanzada para académicos y estudiantes en sociología y planificación territorial. Su énfasis en la seguridad y privacidad, junto con su construcción basada en fuentes de código abierto, lo convierte en un recurso valioso y confiable para la investigación y el aprendizaje.


In [2]:
!pip install -q llama-index chromadb

In [31]:
import os
import chromadb
import requests
from getpass import getpass
from llama_index.llms import OpenAI, Gemini
from llama_index import SimpleDirectoryReader, VectorStoreIndex
from llama_index.response.notebook_utils import display_response
from llama_index.vector_stores import ChromaVectorStore
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.storage.storage_context import StorageContext
from llama_index.service_context import ServiceContext
from llama_index.prompts import PromptTemplate

Setea tu clave de Google

## Ingreso de Google API Key

Para comenzar a interactuar con el **ArguBot**, es necesario que ingreses tu **Google API Key**. Por favor, introduce tu clave en el campo de entrada a continuación.

Recuerda que utilizamos `getpass` para asegurar que tu clave se mantenga **totalmente secreta** y **segura**. Con `getpass`, tu clave **no se almacena** y se mantiene confidencial dentro de la sesión actual del usuario. Tu privacidad y seguridad son nuestra máxima prioridad.

Por favor, ingresa tu Google API Key aquí:


In [17]:
GOOGLE_API_KEY = getpass('Enter your Gogle secret API Key: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter your Gogle secret API Key: ··········


In [16]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [32]:
with requests.get('https://drive.google.com/file/d/1iFHzW_QO_f6LJzBiOB5ZM7taxAvf7Jgr/view?usp=drive_link', stream=True) as r:
    r.raise_for_status()
    with open('/content/src/data.txt', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)


In [23]:
### Cargando documentos
documents = SimpleDirectoryReader('src').load_data()

In [20]:
llm = Gemini(temperature=0.2, model="gemini-pro")

In [21]:
db = chromadb.PersistentClient(path="./chroma_db_HF")

# create collection
chroma_collection = db.get_or_create_collection("Tesis_Tomas_Manzur")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [22]:
### Seteando el service context y el storage context cambiando los valores default
svc = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)
stc = StorageContext.from_defaults(vector_store=vector_store)

In [24]:
## Creando un nuevo index
index = VectorStoreIndex.from_documents(
    documents, storage_context=stc, service_context=svc
)

In [25]:
query_engine = index.as_query_engine()

In [26]:
template = (
    "Dado el contexto que te proporcionare responde las preguntas \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español como si fueras sociologo experto en controversias sociotécnicas y en análisis de conflictos sociales, políticos, ambientales y territoriales: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [27]:
response = query_engine.query('Qué dice Tomás Manzur en su tesis doctoral?')
display_response(response)

**`Final Response:`** En su tesis doctoral, Tomás Manzur analiza el proceso de elaboración y discusión del Plan de Ordenamiento Territorial de la Provincia de Mendoza, con el objetivo de desentrañar el conflicto que subyació a la discusión de esa política de ordenamiento del territorio, reconociendo el rol de los argumentos movilizados durante ese proceso desarrollado entre 2012 y 2017.

Manzur sostiene que el conflicto se debió a la tensión entre diferentes territorialidades, es decir, a la disputa por el control y uso del territorio por parte de distintos actores sociales. Esta tensión se manifestó en la discusión sobre el agua, un recurso escaso y vital en Mendoza.

El autor analiza las controversias socio-técnicas que tuvieron lugar durante la elaboración del Plan, y observa que los actores sociales involucrados en cada uno de los aspectos controvertidos movilizaron diferentes argumentos para defender sus posiciones. Estos argumentos se basaban en diferentes concepciones del territorio y del agua, y en diferentes intereses y valores.

Manzur concluye que el conflicto que subyació a la discusión del Plan de Ordenamiento Territorial de Mendoza fue un conflicto socio-técnico, en el que se enfrentaron diferentes territorialidades y diferentes concepciones del territorio y del agua. Este conflicto se manifestó en la movilización de diferentes argumentos por parte de los actores sociales involucrados, y en la tensión entre los intereses y valores de estos actores.

In [28]:
i = 0
while i < 10:
  inp = input("User: ")
  response = query_engine.query(inp)
  display_response(response)
  i + 1

User: Qué actores participan de la discusión del PPOT?


**`Final Response:`** En la discusión del PPOT participan diversos actores sociales, entre ellos:

- Organismos gubernamentales: Directora de la APOT, Superintendente del Departamento General de Irrigación (DGI) y ex Director del Fondo Para la Transformación y el Crecimiento, Secretario de Gestión Hídrica, Diputado que impulsó el proyecto de Ley de Ordenamiento Territorial y Usos del Suelo, responsable de Relaciones Institucionales de la Agencia de Inversiones Promendoza, Ministro de Tierra, Ambiente y Recursos Naturales, Secretario de Ambiente y Ordenamiento territorial, y los dos gobernadores provinciales que fueron funcionarios durante la elaboración, discusión y aprobación del PPOT.

- Instituciones científico-técnicas: Rector de la UNCuyo, Directora del CIFOT, referentes de El Instituto Nacional del Agua (INA), del Servicio Geológico Minero Argentino (SEGEMAR), del INTA, del Instituto de Ciencias Ambientales (ICA), del Instituto de Desarrollo Rural (IDR), de la Red Integral para la Gestión del Agua (RIGA), de la Coordinadora del Equipo técnico del PED Mendoza 2030, del INCIHUSA, de la Universidad de Congreso, y del Proyecto de Investigación y Desarrollo “Ordenamiento Territorial para un Desarrollo Sostenible” de los Institutos Multidisciplinarios de la Universidad Nacional de Cuyo.

- Organizaciones de empresarios y profesionales: 8 casos de 13.

- Colectivos, asambleas y otras organizaciones de la sociedad civil: 13 casos de 18.

User: Todos los actores sociales analizados en la tesis de Tomás Manzur participaron del PPOT mediante instancias institucionales formales?


**`Final Response:`** No, no todos los actores sociales analizados en la tesis de Tomás Manzur participaron del PPOT mediante instancias institucionales formales.

El autor menciona que la unidad de análisis está conformada por la discursividad de los actores sociales que tomaron parte en la discusión del PPOT, tanto mediante canales institucionalmente dispuestos para el debate como por fuera de ellos.

Esto significa que algunos actores sociales participaron del PPOT a través de instancias institucionales formales, como el Consejo Provincial de Ordenamiento Territorial (CPOT), mientras que otros participaron por fuera de estas instancias, como a través de notas y entrevistas en medios de comunicación, comunicados oficiales publicados en redes sociales/blogs de las organizaciones, etc.

User: Cómo se conectan las controversias sociotécnicas con el PPOT y con el agua?


**`Final Response:`** Las controversias sociotécnicas están relacionadas con el PPOT (Plan Provincial de Ordenamiento Territorial) y el agua de varias maneras:

* **El PPOT es un instrumento de planificación territorial que tiene como objetivo ordenar el uso del suelo y los recursos naturales de la provincia de Mendoza.** Las controversias sociotécnicas pueden surgir cuando diferentes actores sociales tienen intereses contrapuestos sobre el uso del suelo y los recursos naturales, como por ejemplo, cuando se propone la construcción de una nueva represa o la explotación de un yacimiento minero.
* **El agua es un recurso natural esencial para la vida y el desarrollo humano.** Las controversias sociotécnicas pueden surgir cuando diferentes actores sociales tienen intereses contrapuestos sobre el uso del agua, como por ejemplo, cuando se propone la construcción de un nuevo canal de riego o la explotación de un acuífero.
* **El PPOT puede ser una herramienta para gestionar las controversias sociotécnicas relacionadas con el agua.** Al establecer normas y regulaciones sobre el uso del suelo y los recursos naturales, el PPOT puede ayudar a prevenir o mitigar los conflictos entre diferentes actores sociales. Por ejemplo, el PPOT puede establecer zonas de protección ambiental donde no se permite la construcción de nuevas represas o la explotación de yacimientos mineros.

En resumen, las controversias sociotécnicas están relacionadas con el PPOT y el agua de varias maneras. El PPOT puede ser una herramienta para gestionar las controversias sociotécnicas relacionadas con el agua, pero también puede ser una fuente de controversias sociotécnicas si no se tiene en cuenta los intereses de todos los actores sociales involucrados.

KeyboardInterrupt: Interrupted by user